In [ ]:
!pip install rdkit xgboost -q
print("✅ Installation complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 68.3 MB/s eta 0:00:00
✅ Installation complete


In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import xgboost as xgb
from rdkit import RDLogger
from tqdm import tqdm
import warnings
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix
)
import time
import pickle

# Suppress warnings
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

print("✅ Imports complete")

✅ Imports complete


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # CONFIGURATION
BASE_PATH = '/content/drive/MyDrive/XGBoost/'
SMILES_FILE = '/content/drive/MyDrive/XGBoost/Drugs_with_Smiles.csv'

# XGBoost Configuration (Matching HGNN)
config = {
    'max_depth': 8,
    'learning_rate': 0.005,  # Same as HGNN
    'n_estimators': 200,
    'tree_method': 'hist',  # Fast histogram-based algorithm
    'device': 'cpu',
    'random_state': 42,
    'n_jobs': -1  # Use all CPU cores
}

print("Configuration loaded")
print(f"   Base path: {BASE_PATH}")
print(f"   Device: {config['device']} (matching HGNN)")


In [ ]:
# LOAD DATA
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

# Load train/val/test splits
train_df = pd.read_csv(f'{BASE_PATH}data/train_positive.csv')
val_df = pd.read_csv(f'{BASE_PATH}data/val_positive.csv')
test_df = pd.read_csv(f'{BASE_PATH}data/test_positive.csv')

# Load SMILES
smiles_df = pd.read_csv(SMILES_FILE)
smiles_dict = dict(zip(smiles_df['DrugBank_ID'], smiles_df['SMILES']))

print(f"Data loaded:")
print(f"   Training samples: {len(train_df):,}")
print(f"   Validation samples: {len(val_df):,}")
print(f"   Test samples: {len(test_df):,}")
print(f"   Unique drugs: {len(smiles_dict):,}")
print(f"   Interaction types: {train_df['Label'].nunique()} (classes 0-85)")

# Check class distribution
print(f"\nTraining set class distribution:")
class_counts = train_df['Label'].value_counts().sort_index()
print(f"   Min samples per class: {class_counts.min()}")
print(f"   Max samples per class: {class_counts.max()}")
print(f"   Mean samples per class: {class_counts.mean():.1f}")
print(f"   Median samples per class: {class_counts.median():.1f}")


LOADING DATA
Data loaded:
   Training samples: 153,489
   Validation samples: 19,188
   Test samples: 19,200
   Unique drugs: 1,709
   Interaction types: 86 (classes 0-85)

Training set class distribution:
   Min samples per class: 4
   Max samples per class: 48746
   Mean samples per class: 1784.8
   Median samples per class: 227.5


## We use Morgan Fingerprint (2, 2048)

In [ ]:
# COMPUTE DRUG FINGERPRINTS
print("\n" + "="*80)
print("COMPUTING MORGAN FINGERPRINTS")
print("="*80)

def smiles_to_fingerprint(smiles, radius=2, n_bits=2048):
    """Convert SMILES to Morgan fingerprint"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits)
    return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius, n_bits))

# Pre-compute all drug fingerprints
print("Pre-computing fingerprints for all drugs...")
drug_fps = {}
failed_drugs = []

for drug_id, smiles in tqdm(smiles_dict.items(), desc="Computing fingerprints"):
    fp = smiles_to_fingerprint(smiles)
    if fp.sum() == 0:  # Failed to parse
        failed_drugs.append(drug_id)
    drug_fps[drug_id] = fp

print(f"Pre-computed {len(drug_fps)} drug fingerprints")
if failed_drugs:
    print(f"Failed to parse {len(failed_drugs)} SMILES")

print(f"\nFingerprint statistics:")
fp_densities = [fp.sum() / len(fp) for fp in drug_fps.values()]
print(f"   Fingerprint size: 2048 bits")
print(f"   Average density: {np.mean(fp_densities):.2%}")
print(f"   Pair feature size: {2048 * 2} dimensions")


COMPUTING MORGAN FINGERPRINTS
Pre-computing fingerprints for all drugs...


Computing fingerprints: 100%|██████████| 1709/1709 [00:02<00:00, 750.72it/s]

Pre-computed 1709 drug fingerprints
Failed to parse 1 SMILES

Fingerprint statistics:
   Fingerprint size: 2048 bits
   Average density: 2.15%
   Pair feature size: 4096 dimensions


In [ ]:
print("\n" + "="*80)
print("PREPARING DATASETS")
print("="*80)

def prepare_data(df, drug_fps, desc="Processing"):
    """Convert drug pairs to concatenated fingerprints"""
    X = []
    y = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        drug1_fp = drug_fps.get(row['Drug1_ID'], np.zeros(2048))
        drug2_fp = drug_fps.get(row['Drug2_ID'], np.zeros(2048))

        # Concatenate features
        pair_features = np.concatenate([drug1_fp, drug2_fp])

        X.append(pair_features)
        y.append(row['Label'])  # Interaction type (0-85)

    return np.array(X), np.array(y)

# Prepare all datasets
print("\nPreparing training data...")
X_train, y_train = prepare_data(train_df, drug_fps, "Train")

print("\nPreparing validation data...")
X_val, y_val = prepare_data(val_df, drug_fps, "Validation")

print("\nPreparing test data...")
X_test, y_test = prepare_data(test_df, drug_fps, "Test")
y_train = y_train - 1  # Convert 1-86 to 0-85
y_val = y_val - 1
y_test = y_test - 1
print(f"\nData preparation complete:")
print(f"   X_train shape: {X_train.shape}")
print(f"   X_val shape: {X_val.shape}")
print(f"   X_test shape: {X_test.shape}")
print(f"   Number of classes: {len(np.unique(y_train))}")

# Memory usage
train_size_mb = X_train.nbytes / (1024**2)
val_size_mb = X_val.nbytes / (1024**2)
test_size_mb = X_test.nbytes / (1024**2)
print(f"\nMemory usage:")
print(f"   Training: {train_size_mb:.2f} MB")
print(f"   Validation: {val_size_mb:.2f} MB")
print(f"   Test: {test_size_mb:.2f} MB")
print(f"   Total: {train_size_mb + val_size_mb + test_size_mb:.2f} MB")


PREPARING DATASETS

Preparing training data...


Train: 100%|██████████| 153489/153489 [00:10<00:00, 14548.25it/s]



Preparing validation data...


Validation: 100%|██████████| 19188/19188 [00:00<00:00, 21537.20it/s]



Preparing test data...


Test: 100%|██████████| 19200/19200 [00:00<00:00, 21406.77it/s]



Data preparation complete:
   X_train shape: (153489, 4096)
   X_val shape: (19188, 4096)
   X_test shape: (19200, 4096)
   Number of classes: 86

Memory usage:
   Training: 4796.53 MB
   Validation: 599.62 MB
   Test: 600.00 MB
   Total: 5996.16 MB


## We use class weight

In [ ]:
print("\n" + "="*80)
print("CALCULATING CLASS WEIGHTS")
print("="*80)

# Count samples per class
type_counts = np.bincount(y_train, minlength=86)

# Calculate weights using same formula as HGNN
alpha = 0.3  # Same as HGNN
class_weights = 1.0 / np.power(np.maximum(type_counts, 1), alpha)
class_weights = class_weights / class_weights.mean()

print(f"\nClass weight statistics:")
print(f"   Alpha parameter: {alpha}")
print(f"   Min weight: {class_weights.min():.4f}")
print(f"   Max weight: {class_weights.max():.4f}")
print(f"   Mean weight: {class_weights.mean():.4f}")
print(f"\nSample distribution:")
print(f"   Min samples: {type_counts.min()}")
print(f"   Max samples: {type_counts.max()}")
print(f"   Mean samples: {type_counts.mean():.1f}")
print(f"   Imbalance ratio: {type_counts.max() / max(type_counts.min(), 1):.1f}x")

# Convert to sample weights for XGBoost
sample_weights = class_weights[y_train]
print(f"\nSample weights prepared for {len(sample_weights):,} training samples")


CALCULATING CLASS WEIGHTS

Class weight statistics:
   Alpha parameter: 0.3
   Min weight: 0.1566
   Max weight: 2.6340
   Mean weight: 1.0000

Sample distribution:
   Min samples: 4
   Max samples: 48746
   Mean samples: 1784.8
   Imbalance ratio: 12186.5x

Sample weights prepared for 153,489 training samples


In [ ]:
print("\n" + "="*80)
print("TRAINING XGBOOST MODEL (UNWEIGHTED)")
print("="*80)

print(f"\nConfiguration:")
for key, value in config.items():
    print(f"   {key}: {value}")

# Create XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=86,
    max_depth=config['max_depth'],
    learning_rate=config['learning_rate'],
    n_estimators=config['n_estimators'],
    tree_method=config['tree_method'],
    device=config['device'],
    random_state=config['random_state'],
    n_jobs=config['n_jobs'],
    eval_metric='mlogloss'
)

print(f"\nTraining configuration:")
print(f"   Training on: X_train only ({len(X_train):,} samples)")
print(f"   Monitoring on: X_val ({len(X_val):,} samples) - for progress tracking only")
print(f"   Final evaluation: X_test ({len(X_test):,} samples)")
print(f"   Sample weighting: None (unweighted training)")

print(f"\nStarting training with {config['n_estimators']} trees...")
start_time = time.time()

# Train WITHOUT sample weights - validation used for monitoring only
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],  # Validation for monitoring only, NOT for training
    verbose=True
)

train_time = time.time() - start_time

print(f"\nTraining complete!")
print(f"   Training time: {train_time/60:.2f} minutes ({train_time:.1f} seconds)")
print(f"   Number of trees: {xgb_model.n_estimators}")
print(f"   Max depth: {xgb_model.max_depth}")
print(f"   Trained on: {len(X_train):,} samples (X_train only)")


TRAINING XGBOOST MODEL (UNWEIGHTED)

Configuration:
   max_depth: 8
   learning_rate: 0.005
   n_estimators: 200
   tree_method: hist
   device: cpu
   random_state: 42
   n_jobs: -1

Training configuration:
   Training on: X_train only (153,489 samples)
   Monitoring on: X_val (19,188 samples) - for progress tracking only
   Final evaluation: X_test (19,200 samples)
   Sample weighting: None (unweighted training)

Starting training with 200 trees...
[0]	validation_0-mlogloss:4.24837
[1]	validation_0-mlogloss:4.19273
[2]	validation_0-mlogloss:4.14114
[3]	validation_0-mlogloss:4.09306
[4]	validation_0-mlogloss:4.04754
[5]	validation_0-mlogloss:4.00444
[6]	validation_0-mlogloss:3.96362
[7]	validation_0-mlogloss:3.92399
[8]	validation_0-mlogloss:3.88647
[9]	validation_0-mlogloss:3.85083
[10]	validation_0-mlogloss:3.81604
[11]	validation_0-mlogloss:3.78304
[12]	validation_0-mlogloss:3.74932
[13]	validation_0-mlogloss:3.71716
[14]	validation_0-mlogloss:3.68734
[15]	validation_0-mlogloss:3.

System RAM
11.2 / 12.7 GB

**this from side window of google colab**

## 7. Evaluate on Validation Set

In [ ]:
# ============================================================================
# TEST SET EVALUATION
# ============================================================================
print("\n" + "="*80)
print("TEST SET EVALUATION - FINAL RESULTS")
print("="*80)

print("\n🔍 Making predictions on test set...")
test_start = time.time()

# Get predictions and probabilities
y_test_pred = xgb_model.predict(X_test)
y_test_proba = xgb_model.predict_proba(X_test)  # For ROC-AUC and Top-K

test_time = time.time() - test_start

# ============================================================================
# STANDARD METRICS (Macro & Weighted)
# ============================================================================
test_accuracy = accuracy_score(y_test, y_test_pred)

test_f1_macro = f1_score(y_test, y_test_pred, average='macro')
test_f1_weighted = f1_score(y_test, y_test_pred, average='weighted')

test_precision_macro = precision_score(y_test, y_test_pred, average='macro')
test_precision_weighted = precision_score(y_test, y_test_pred, average='weighted')

test_recall_macro = recall_score(y_test, y_test_pred, average='macro')
test_recall_weighted = recall_score(y_test, y_test_pred, average='weighted')

# ============================================================================
# ROC-AUC (One-vs-Rest)
# ============================================================================
from sklearn.metrics import roc_auc_score
test_roc_auc_macro = roc_auc_score(y_test, y_test_proba, average='macro', multi_class='ovr')
test_roc_auc_weighted = roc_auc_score(y_test, y_test_proba, average='weighted', multi_class='ovr')

# ============================================================================
# TOP-K ACCURACY
# ============================================================================
def top_k_accuracy(y_true, y_proba, k):
    """Calculate top-k accuracy"""
    top_k_preds = np.argsort(y_proba, axis=1)[:, -k:]  # Get top-k predictions
    correct = np.array([y_true[i] in top_k_preds[i] for i in range(len(y_true))])
    return np.mean(correct)

top1_accuracy = top_k_accuracy(y_test, y_test_proba, k=1)  # Same as regular accuracy
top3_accuracy = top_k_accuracy(y_test, y_test_proba, k=3)

# ============================================================================
# DISPLAY RESULTS
# ============================================================================
print(f"\n" + "="*80)
print("🎯 XGBOOST - FINAL TEST RESULTS (Use for GNN Comparison)")
print("="*80)

print(f"\n📊 ACCURACY METRICS:")
print(f"   Top-1 Accuracy:          {test_accuracy:.4f}")
print(f"   Top-3 Accuracy:          {top3_accuracy:.4f}")

print(f"\n📊 F1-SCORE:")
print(f"   F1-Macro:                {test_f1_macro:.4f}")
print(f"   F1-Weighted:             {test_f1_weighted:.4f}")

print(f"\n📊 PRECISION:")
print(f"   Precision-Macro:         {test_precision_macro:.4f}")
print(f"   Precision-Weighted:      {test_precision_weighted:.4f}")

print(f"\n📊 RECALL:")
print(f"   Recall-Macro:            {test_recall_macro:.4f}")
print(f"   Recall-Weighted:         {test_recall_weighted:.4f}")

print(f"\n📊 ROC-AUC:")
print(f"   ROC-AUC-Macro:           {test_roc_auc_macro:.4f}")
print(f"   ROC-AUC-Weighted:        {test_roc_auc_weighted:.4f}")

print(f"\n⏱️  PERFORMANCE:")
print(f"   Prediction time:         {test_time:.2f} seconds")
print(f"   Test samples:            {len(X_test):,}")

print(f"\n" + "="*80)
print(f"✅ These are the final metrics for comparing with our GNN model")
print("="*80)


TEST SET EVALUATION - FINAL RESULTS

🔍 Making predictions on test set...

🎯 XGBOOST - FINAL TEST RESULTS (Use for GNN Comparison)

📊 ACCURACY METRICS:
   Top-1 Accuracy:          0.6955
   Top-3 Accuracy:          0.9224

📊 F1-SCORE:
   F1-Macro:                0.5497
   F1-Weighted:             0.6791

📊 PRECISION:
   Precision-Macro:         0.6665
   Precision-Weighted:      0.7307

📊 RECALL:
   Recall-Macro:            0.5076
   Recall-Weighted:         0.6955

📊 ROC-AUC:
   ROC-AUC-Macro:           0.9709
   ROC-AUC-Weighted:        0.9457

⏱️  PERFORMANCE:
   Prediction time:         21.98 seconds
   Test samples:            19,200

✅ These are the final metrics for comparing with your GNN model


## 8. Test Set Evaluation

In [ ]:
# ============================================================================
# TEST SET EVALUATION
# ============================================================================
print("\n" + "="*80)
print("TEST SET EVALUATION")
print("="*80)

print("\n🔍 Making predictions on test set...")
test_start = time.time()
y_test_pred = xgb_model.predict(X_test)
test_time = time.time() - test_start

# Calculate all metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1_micro = f1_score(y_test, y_test_pred, average='micro')
test_f1_macro = f1_score(y_test, y_test_pred, average='macro')
test_precision = precision_score(y_test, y_test_pred, average='micro')
test_recall = recall_score(y_test, y_test_pred, average='micro')

print(f"\n" + "="*80)
print("XGBOOST - FINAL TEST RESULTS")
print("="*80)
print(f"\n📊 Test Set Performance:")
print(f"   Accuracy:     {test_accuracy:.4f}")
print(f"   Precision:    {test_precision:.4f}")
print(f"   Recall:       {test_recall:.4f}")
print(f"   F1-Micro:     {test_f1_micro:.4f}")
print(f"   F1-Macro:     {test_f1_macro:.4f}")
print(f"\n⏱️  Timing:")
print(f"   Training time:    {train_time/60:.2f} minutes")
print(f"   Inference time:   {test_time:.2f} seconds")
print(f"   Speed: {len(y_test)/test_time:.1f} predictions/sec")
print("="*80)


TEST SET EVALUATION

🔍 Making predictions on test set...

XGBOOST - FINAL TEST RESULTS

📊 Test Set Performance:
   Accuracy:     0.6955
   Precision:    0.6955
   Recall:       0.6955
   F1-Micro:     0.6955
   F1-Macro:     0.5497

⏱️  Timing:
   Training time:    173.71 minutes
   Inference time:   10.82 seconds
   Speed: 1774.2 predictions/sec


## 9. Detailed Performance Analysis

In [ ]:
# ============================================================================
# PER-CLASS PERFORMANCE ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("PER-CLASS PERFORMANCE ANALYSIS")
print("="*80)

# Calculate per-class metrics
per_class_f1 = f1_score(y_test, y_test_pred, average=None)
per_class_precision = precision_score(y_test, y_test_pred, average=None, zero_division=0)
per_class_recall = recall_score(y_test, y_test_pred, average=None, zero_division=0)

# Find best and worst performing classes
best_classes = np.argsort(per_class_f1)[-5:][::-1]
worst_classes = np.argsort(per_class_f1)[:5]

print(f"\nTop 5 Best Performing Classes:")
for i, class_idx in enumerate(best_classes, 1):
    test_samples = (y_test == class_idx).sum()
    print(f"   {i}. Class {class_idx}: F1={per_class_f1[class_idx]:.4f}, "
          f"Precision={per_class_precision[class_idx]:.4f}, "
          f"Recall={per_class_recall[class_idx]:.4f} ({test_samples} samples)")

print(f"\nBottom 5 Worst Performing Classes:")
for i, class_idx in enumerate(worst_classes, 1):
    test_samples = (y_test == class_idx).sum()
    print(f"   {i}. Class {class_idx}: F1={per_class_f1[class_idx]:.4f}, "
          f"Precision={per_class_precision[class_idx]:.4f}, "
          f"Recall={per_class_recall[class_idx]:.4f} ({test_samples} samples)")

print(f"\nOverall Statistics:")
print(f"   Mean F1 across classes: {per_class_f1.mean():.4f}")
print(f"   Std F1 across classes: {per_class_f1.std():.4f}")
print(f"   Classes with F1 > 0.8: {(per_class_f1 > 0.8).sum()}/86")
print(f"   Classes with F1 < 0.5: {(per_class_f1 < 0.5).sum()}/86")


PER-CLASS PERFORMANCE ANALYSIS

Top 5 Best Performing Classes:
   1. Class 6: F1=1.0000, Precision=1.0000, Recall=1.0000 (1 samples)
   2. Class 0: F1=1.0000, Precision=1.0000, Recall=1.0000 (2 samples)
   3. Class 44: F1=1.0000, Precision=1.0000, Recall=1.0000 (2 samples)
   4. Class 42: F1=1.0000, Precision=1.0000, Recall=1.0000 (1 samples)
   5. Class 37: F1=1.0000, Precision=1.0000, Recall=1.0000 (2 samples)

Bottom 5 Worst Performing Classes:
   1. Class 25: F1=0.0000, Precision=0.0000, Recall=0.0000 (1 samples)
   2. Class 27: F1=0.0000, Precision=0.0000, Recall=0.0000 (1 samples)
   3. Class 43: F1=0.0000, Precision=0.0000, Recall=0.0000 (2 samples)
   4. Class 41: F1=0.0000, Precision=0.0000, Recall=0.0000 (1 samples)
   5. Class 51: F1=0.0000, Precision=0.0000, Recall=0.0000 (1 samples)

Overall Statistics:
   Mean F1 across classes: 0.6429
   Std F1 across classes: 0.2562
   Classes with F1 > 0.8: 25/86
   Classes with F1 < 0.5: 16/86


## 10. Save Model & Results

In [ ]:
# ============================================================================
# SAVE MODEL AND RESULTS
# ============================================================================
print("\n" + "="*80)
print("SAVING MODEL AND RESULTS")
print("="*80)

# Save XGBoost model
model_path = f'{BASE_PATH}xgboost_model.json'
xgb_model.save_model(model_path)
print(f"✅ Model saved to: {model_path}")

# Save results
results = {
    'config': config,
    'train_time_minutes': train_time / 60,
    'test_time_seconds': test_time,
    'validation_metrics': {
        'accuracy': val_accuracy,
        'f1_micro': val_f1_micro,
        'f1_macro': val_f1_macro,
        'precision': val_precision,
        'recall': val_recall
    },
    'test_metrics': {
        'accuracy': test_accuracy,
        'f1_micro': test_f1_micro,
        'f1_macro': test_f1_macro,
        'precision': test_precision,
        'recall': test_recall
    },
    'per_class_f1': per_class_f1.tolist(),
    'class_weights': class_weights.tolist(),
    'predictions': {
        'y_test_true': y_test.tolist(),
        'y_test_pred': y_test_pred.tolist()
    }
}

results_path = f'{BASE_PATH}xgboost_results.pkl'
with open(results_path, 'wb') as f:
    pickle.dump(results, f)
print(f"✅ Results saved to: {results_path}")

# Save summary as CSV
summary_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Micro', 'F1-Macro'],
    'Validation': [val_accuracy, val_precision, val_recall, val_f1_micro, val_f1_macro],
    'Test': [test_accuracy, test_precision, test_recall, test_f1_micro, test_f1_macro]
})

summary_path = f'{BASE_PATH}xgboost_summary.csv'
summary_df.to_csv(summary_path, index=False)
print(f"✅ Summary saved to: {summary_path}")

print(f"\n" + "="*80)
print("ALL DONE! ✨")
print("="*80)


SAVING MODEL AND RESULTS
✅ Model saved to: /content/drive/MyDrive/XGBoost/xgboost_model.json
✅ Results saved to: /content/drive/MyDrive/XGBoost/xgboost_results.pkl
✅ Summary saved to: /content/drive/MyDrive/XGBoost/xgboost_summary.csv

ALL DONE! ✨
